# HuggingFace TrieGuided Decoding

In [1]:
# Open-Orca/Mistral-7B-OpenOrca

In [2]:
torch_device = 'cpu'

In [3]:
!pip install outlines -q
!pip install -q transformers 
!pip install -q openai 
!pip install -q datasets
!pip install -q accelerate
!pip install trieregex

  Using cached trieregex-1.0.0-py3-none-any.whl


In [4]:
import site
import os
import transformers  # replace 'my_library' with the actual library name

library_path = os.path.dirname(transformers.__file__)
print(library_path)


/opt/.cs/conda/envs/codeserver_py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/opt/.cs/conda/envs/codeserver_py39/lib/python3.9/site-packages/transformers


In [ ]:
!jupyterlab --ContentsManager.allow_hidden=True


/bin/bash: line 1: jupyterlab: command not found


: 

In [ ]:
import sys
print(sys.path)

['/root/projects/lexflow/src/notebooks', '/opt/conda/lib/python38.zip', '/opt/conda/lib/python3.8', '/opt/conda/lib/python3.8/lib-dynload', '', '/opt/conda/lib/python3.8/site-packages', '/opt/conda/lib/python3.8/site-packages/smdebug-1.0.13b20220304-py3.8.egg', '/opt/conda/lib/python3.8/site-packages/pyinstrument-3.4.2-py3.8.egg', '/opt/conda/lib/python3.8/site-packages/pyinstrument_cext-0.2.4-py3.8-linux-x86_64.egg', '/opt/conda/lib/python3.8/site-packages/IPython/extensions', '/root/.ipython']


In [ ]:
!python -version


In [ ]:
print('hello')

## Exploring api

In [2]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


ModuleNotFoundError: No module named 'transformers'

In [4]:
model.generation_config

GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256
}

In [6]:
from transformers import LogitsProcessor

class CustomLogitsProcessor(LogitsProcessor):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor) -> torch.FloatTensor:
        # Modify the scores here according to your custom function
        return modified_scores

In [10]:
from transformers.generation.logits_process import LogitsProcessorList

logit_processors = LogitsProcessorList([CustomLogitsProcessor()])

In [11]:
input_ids = tokenizer('I enjoy walking with my cute dog', return_tensors='pt').to(torch_device)

In [17]:
!pip freeze | grep transformers

transformers==4.35.0


In [13]:
model.generate(**input_ids, logit_processors=logit_processors)

ValueError: The following `model_kwargs` are not used by the model: ['logit_processors'] (note: typos in the generate arguments will also show up in this list)

In [18]:
model_inputs = tokenizer('I enjoy walking with my cute dog', return_tensors='pt').to(torch_device)
sample_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=50,
    num_beams=5,
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I also like to play with my dog.

How do you feel about your dog?

I love to play with my dog. I love to play with my dog. I


In [19]:
sample_output.shape

torch.Size([1, 47])

## testing huggingface

In [1]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Encode the input text
input_text = "The current weather is"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Get the model's output (logits)
with torch.no_grad():
    outputs = model(input_ids)
    predictions = outputs[0]

# Apply softmax to convert logits to probabilities
softmax = torch.nn.Softmax(dim=-1)
probabilities = softmax(predictions)

# Iterate through each word in the input and check the prediction for the next word
for i in range(len(input_ids[0]) - 1):
    # Get the index of the actual next word
    actual_next_word_id = input_ids[0][i+1].item()
    # Get the probability distribution for the next word
    next_word_probabilities = probabilities[0, i]
    # Get the predicted token ID with the highest probability for the next word
    predicted_next_word_id = next_word_probabilities.argmax().item()
    
    # Decode the predicted and actual next words
    predicted_token = tokenizer.decode([predicted_next_word_id])
    actual_next_word = tokenizer.decode([actual_next_word_id])
    
    print(f"Input word: '{tokenizer.decode([input_ids[0][i].item()])}'")
    print(f"Predicted next word: '{predicted_token}'")
    print(f"Actual next word: '{actual_next_word}'")
    print()

# Now handle the last token separately since there's no actual next word in the input
last_word_probabilities = probabilities[0, -1]
predicted_next_word_id_for_last = last_word_probabilities.argmax().item()
predicted_token_for_last = tokenizer.decode([predicted_next_word_id_for_last])

print(f"Input word: '{tokenizer.decode([input_ids[0][-1].item()])}'")
print(f"Predicted next word: '{predicted_token_for_last}'")


[2023-11-15 13:23:06.045 pytorch-1-10-cpu-py38-ml-t3-medium-7bde6515cda8eba7f7394af8b8e0:1272 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None


/opt/conda/lib/python3.8/site-packages/smdebug-1.0.13b20220304-py3.8.egg/smdebug/profiler/system_metrics_reader.py:63: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/opt/conda/lib/python3.8/site-packages/smdebug-1.0.13b20220304-py3.8.egg/smdebug/profiler/system_metrics_reader.py:63: SyntaxWarning: "is not" with a literal. Did you mean "!="?


[2023-11-15 13:23:06.403 pytorch-1-10-cpu-py38-ml-t3-medium-7bde6515cda8eba7f7394af8b8e0:1272 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Input word: 'The'
Predicted next word: '
'
Actual next word: ' current'

Input word: ' current'
Predicted next word: ' state'
Actual next word: ' weather'

Input word: ' weather'
Predicted next word: ' is'
Actual next word: ' is'

Input word: ' is'
Predicted next word: ' not'


In [2]:
probabilities.shape

torch.Size([1, 4, 50257])

In [3]:
# Get the logits for the last token in the input
last_token_logits = predictions[:, -1, :]

# Define candidate words
candidate_words = ["sunny", "rainy", "cloudy", "windy"]

# Retrieve the indices of the candidate words
candidate_indices = [tokenizer.encode(word)[0] for word in candidate_words]

# Extract the logits for the candidate words
candidate_logits = last_token_logits[:, candidate_indices]

# Find the most likely candidate word
most_likely_word_index = candidate_logits.argmax(-1).item()
predicted_word = candidate_words[most_likely_word_index]

print(predicted_word)

sunny


In [4]:
most_likely_word_index

0

In [5]:
candidate_logits.shape

torch.Size([1, 4])

## Using it on document

In [6]:
def read_document(file_path):
    """
    Reads and returns the contents of a text file.

    Parameters:
    file_path (str): The path to the text file to be read.

    Returns:
    str: The contents of the file.
    """
    try:
        with open(file_path, 'r') as file:
            return file.read()
    except FileNotFoundError:
        return "File not found."
    except Exception as e:
        return str(e)

# Example usage:
document = read_document('document.txt')
# print(content)


In [7]:
document[:10]

'Suella Bra'

In [67]:
from collections import defaultdict

def build_word_graph(document):
    # Using defaultdict to automatically handle missing keys
    graph = defaultdict(set)
    previous_word = None
    # Loop over each word in the document
    for word in document.split():
        # Skip adding the first word to any previous word
        if previous_word is not None:
            # Add the word to the set of words that come after the previous word
            graph[previous_word].add(word)
        previous_word = word  # Update the previous word
    return graph

# Example usage
# document = "hello my name is hello his name is ...."
word_graph = build_word_graph(document)

# Now you can query the graph for what comes after "hello"
print(word_graph["Braverman"])  # Outputs: {'my', 'his'}


{'and', 'claim', 'accuses', 'for', '-', 'said', 'was', 'has'}


In [82]:
class TrieNode:
    def __init__(self):
        self.children = defaultdict(TrieNode)
        self.next_words = defaultdict(int)  # Stores next words and their frequencies

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, tokens):
        # Insert all subsequences of tokens into the trie
        for i in range(len(tokens)):
            current = self.root
            for j in range(i, len(tokens)):
                current = current.children[tokens[j]]
                if j + 1 < len(tokens):
                    current.next_words[tokens[j + 1]] += 1  # Increment the count of the next word

    def search_next_words(self, sequence):
        # Find the next possible words after the sequence of tokens
        current = self.root
        for token in sequence:
            if token not in current.children:
                return {}  # If the sequence is not found, return an empty dictionary
            current = current.children[token]
        return current.next_words

# Example usage:
trie = Trie()

# Tokenize the text and insert into the trie
# Assuming 'text' is a string containing the full document
# text = """
# hello there how are you
# hello there how have you been
# hello there who are you
# hello there she is my friend
# hello there he is a colleague
# """
text = document
tokens = text.split()
trie.insert(tokens)



In [83]:
# Find the next words after the sequence "hello there"
sequence = "Mrs Braverman said Mr Sunak had".split()
next_words = trie.search_next_words(sequence)
next_words


defaultdict(int, {'failed': 1})

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


In [88]:

# Encode the input text
input_text = "Extract from the text what Mrs Braverman said"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Get the model's output (logits)
with torch.no_grad():
    outputs = model(input_ids)
    predictions = outputs[0]

# Apply softmax to convert logits to probabilities
softmax = torch.nn.Softmax(dim=-1)
probabilities = softmax(predictions)

# Now handle the last token separately since there's no actual next word in the input
last_word_probabilities = probabilities[0, -1]
last_token_logits = predictions[:, -1, :]

# Define the trie with potential next words
# trie = # (Assume this is your trie object which has the method search_next_words(sequence))

# Get the possible next words from the trie given the sequence of tokens
sequence_of_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
next_words_from_trie = trie.search_next_words(sequence_of_tokens)
print(next_words_from_trie)
# Convert these possible next words to their corresponding token IDs
next_word_ids_from_trie = {tokenizer.encode(word)[0] for word in next_words_from_trie}

# Mask the probabilities
masked_probabilities = last_word_probabilities.clone()  # Clone the tensor to avoid modifying the original probabilities
for idx in range(masked_probabilities.size(0)):
    if idx not in next_word_ids_from_trie:
        masked_probabilities[idx] = 0

# Renormalize the probabilities
masked_probabilities /= masked_probabilities.sum()

# Find the most likely word from the trie
most_likely_word_id = masked_probabilities.argmax().item()
predicted_word = tokenizer.decode([most_likely_word_id])

print(predicted_word)


{}
!


In [115]:

# Define the function to predict the next word using a trie for restrictions
def predict_next_word(input_text, trie=None, use_trie=True):
    """
    Predicts the next word using GPT-2 model with an option to restrict predictions based on a trie.
    
    Parameters:
    input_text (str): The input text string for which we want to predict the next word.
    trie (Trie object): An optional trie object for restricting predictions.
    use_trie (bool): A flag to determine whether to use the trie for restricting predictions.
    
    Returns:
    str: The predicted next word.
    """
    # Encode the input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Get the model's output (logits)
    with torch.no_grad():
        outputs = model(input_ids)
        predictions = outputs[0]

    # Apply softmax to convert logits to probabilities
    softmax = torch.nn.Softmax(dim=-1)
    probabilities = softmax(predictions)

    # Get the logits and probabilities for the last token in the input
    last_token_logits = predictions[:, -1, :]
    last_word_probabilities = probabilities[0, -1]

    # If the trie is provided and we're using it to restrict predictions
    if trie and use_trie:
        # Get the sequence of tokens from the input text
        sequence_of_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
        
        # Get the possible next words from the trie given the sequence of tokens
        next_words_from_trie = trie.search_next_words(sequence_of_tokens)
        
        # Convert these possible next words to their corresponding token IDs
        next_word_ids_from_trie = {tokenizer.encode(word, add_prefix_space=True)[0] for word in next_words_from_trie}

        # Mask the probabilities for tokens not in the trie's next words
        mask = torch.full_like(last_word_probabilities, fill_value=float('-inf'))
        for idx in next_word_ids_from_trie:
            mask[idx] = 0
        masked_logits = last_token_logits + mask

        # Apply softmax to convert masked logits to probabilities
        masked_probabilities = softmax(masked_logits)
    else:
        masked_probabilities = last_word_probabilities

    # Get the token ID with the highest probability
    most_likely_word_id = masked_probabilities.argmax().item()
    # Decode the token ID to get the word
    predicted_word = tokenizer.decode([most_likely_word_id])
    
    return predicted_word

# Example usage:

predicted_word = predict_next_word(document[:512] + "Extract from the text what Mrs Braverman said:'", trie, use_trie=True)
print(predicted_word)

!


In [ ]:
# gotta do some proper decoding
# why am i getting ! it should always be one of the words
# maybe there is a need for fine tuning